In [1]:
from sail.models.torch.tcn import TCNRegressor
import numpy as np
import torch
from torch.autograd import Variable

def data_generator(N, seq_length):
    """
    Args:
        seq_length: Length of the adding problem data
        N: # of data in the set
    """
    X_num = torch.rand([N, 1, seq_length])
    X_mask = torch.zeros([N, 1, seq_length])
    Y = torch.zeros([N, 1])
    for i in range(N):
        positions = np.random.choice(seq_length, size=2, replace=False)
        X_mask[i, 0, positions[0]] = 1
        X_mask[i, 0, positions[1]] = 1
        Y[i,0] = X_num[i, 0, positions[0]] + X_num[i, 0, positions[1]]
    X = torch.cat((X_num, X_mask), dim=1)
    return Variable(X), Variable(Y)

In [2]:
input_channels = 2
n_classes = 1
batch_size = 1
seq_length = 400
#epochs = 10
X_train, Y_train = data_generator(100000, seq_length)
X_test, Y_test = data_generator(1000, seq_length)


In [3]:
# Note: We use a very simple setting here (assuming all levels have the same # of channels.
channel_sizes = [30]*8 
kernel_size = 7
dropout = 0.0

learning_rate =4e-3

tcn = TCNRegressor(input_channels,n_classes, layers=channel_sizes, ks=kernel_size, conv_dropout=dropout, fc_dropout=dropout)
#optimizer = torch.optim.Adam(tcn.get_params(), lr=learning_rate)
y_pred= []

for i in range(0,X_train.size(0), batch_size):
    x, y = X_train[i:(i+batch_size)], Y_train[i:(i+batch_size)]
    tcn.partial_fit(x, y)
    y_pred.append(tcn.predict(x)[0][0]) 

  epoch    train_loss     dur
-------  ------------  ------
      1        0.2520  0.0255
      2        0.2659  0.0194
      3        1.9575  0.0212
      4        2.4388  0.0216
      5        0.4355  0.0225
      6        0.7585  0.0222
      7        0.0007  0.0215
      8        0.0728  0.0212
      9        0.3566  0.0210
     10        0.0368  0.0197
     11        0.1717  0.0215
     12        0.0279  0.0205
     13        0.0001  0.0205
     14        0.1169  0.0212
     15        0.0949  0.0197
     16        0.0537  0.0198
     17        0.2039  0.0195
     18        0.7656  0.0205
     19        0.0718  0.0199
     20        0.1912  0.0189
     21        0.0309  0.0194
     22        0.4501  0.0201
     23        0.0023  0.0204
     24        0.0199  0.0189
     25        0.5548  0.0219
     26        0.0689  0.0210
     27        0.0000  0.0200
     28        0.2152  0.0200
     29        0.0456  0.0198
     30        0.0016  0.0192
     31        0.4942  0.0185
     32   

KeyboardInterrupt: 

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio

print(f"loss: {tcn.get_loss(y_pred=torch.tensor(y_pred), y_true=Y_train)}")

pio.renderers.default = 'vscode'
y_true = y.flatten().tolist()
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(x.size(0))), y=y_pred, name='predicted',
                         line=dict(color='red', width=2)))
fig.add_trace(go.Scatter(x=list(range(x.size(0))), y=y_true, name='actual',
                         line=dict(color='blue', width=2)))
fig.show()